In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.datasets import imdb
import numpy as np

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
(X_train_seq, Y_train), (X_test_seq, Y_test) = imdb.load_data()

vocabulary_size = -1

for seq in X_train_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

for seq in X_test_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

vocabulary_size += 1

X_train = sequence.pad_sequences(X_train_seq, maxlen=100)
X_test = sequence.pad_sequences(X_test_seq, maxlen=100)

In [3]:
print(np.shape(X_train_seq), np.shape(Y_train), np.shape(X_test_seq), np.shape(Y_test))
print(np.shape(X_train_seq[0]), np.shape(X_test_seq[0]))
print(np.shape(X_train), np.shape(X_test))

(25000,) (25000,) (25000,) (25000,)
(218,) (68,)
(25000, 100) (25000, 100)


In [4]:
model = Sequential()
model.add(Embedding(output_dim=300, input_dim=vocabulary_size, input_length=100))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, epochs=10, batch_size=128)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          26576100  
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 64)           57664     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3201      
Total params: 26,636,965
Trainable params: 26,636,965
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
196/196 [==============================] - 7s 5ms/step - loss: 0.4493 - accuracy: 0.7802
Epoch 2/10

In [5]:
model.evaluate(X_train,Y_train)

782/782 [==============================] - 2s 2ms/step - loss: 5.3736e-06 - accuracy: 1.0000


[5.3736334848508704e-06, 1.0]

In [6]:
model.evaluate(X_test,Y_test)

782/782 [==============================] - 2s 2ms/step - loss: 1.0043 - accuracy: 0.8395


[1.0042940378189087, 0.8394799828529358]